In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


#### 9~12 Line :  hands 손가락 인식모듈의  작동 option을 지정한다. 
* max_num_hands=1,   : 인식할 손모양의 갯수, 생략하면 2가 지정된다. 
* min_detection_confidence=0.5,  : 성공적인 것으로 간주되는 최소 신뢰도 값. 0.0 ~1.0사이로서 기본값은 0.5이다.
* min_tracking_confidence=0.5 :손 랜드마크가 성공적으로 추적된 것으로 간주되는 최소 신뢰도 값. 0.0 ~1.0 사이로서 기본값은 0.5이다. 이 값을 높이면 시간이 더 소요되지만 좀 더 정확한 작동이 보장된다. 

#### 18 Line :  OpenCV 영상은 BGR 형식인데 MediaPipe에서는 RGB 형식을 사용하므로 영상형식을 변환해 준다  
#### 20 Line : MediaPipe의 hands 모듈을 이용해서 손동작을 인식한다. 이 한줄로서 손동작 인식 AI모델이 작동되고 결과 값이 result로 저장된다. 
#### 22 Line :  MediaPipe용 RGB 형식으로 변환했던 것을 OpenCV 영상저리를위해 다시 BGR형식으로 되돌린다. 
#### 24~25 : result값이 정상인 경우에만 후속 작업 처리한다. 
#### 26~27 : result로 반환된 landmark 데이터를 사용한다. 

인식된 손가락 모양은 다음과 같은 index값으로 가지는 배열로 제공된다.

즉, 업지손가락 끝은 landmark[4]에, 검지 손가락 끝은 landmark[8]에 좌표값이 반환되는데, 좌표값은 image상의 x,y위치값을 0.0~1.0 사이의 값으로 표시한다. 즉 image 좌측 최상단은 x=0.0 y=0.0  우측최하단은 x=1.0, y=1.0이 된다. 따라서int(hand_landmarks.landmark[4].x * 100)은 엄지손가락 끝의 x좌표를 100분율로 표시한 것이 된다. 

#### 28 Line : 두 손가락 끝의 x좌표값 차이의 절대값을 구해 두 손가락 끝이 벌어진 정도를 계산한다. 
##### 34 ~ 35 Line : MediaPipe에 내장된 유틸리티 기능을 이용해서 구해진 손가락 모양을 서로 연결한 그림을 그려준다. 

In [2]:
import cv2
import mediapipe as mp


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
 
cap = cv2.VideoCapture(0)
 
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
 
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
 
        results = hands.process(image)
 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                finger1 = int(hand_landmarks.landmark[4].x * 100 )
                finger2 = int(hand_landmarks.landmark[8].x * 100 )
                dist = abs(finger1 - finger2)
                x = int(hand_landmarks.landmark[4].x * 1280 )
                y = int(hand_landmarks.landmark[4].y * 720 )
                cv2.putText(
                    image, text='f1=%d f2=%d dist=%d ' % (finger1,finger2,dist), org=(10, 30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,
                    color=255, thickness=3)
 
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
 
        cv2.imshow('image', image)
        if cv2.waitKey(1) == ord('q'):
            break
cv2.destroyAllWindows()
cap.release()

2 : 배열이 누적되면 누적될수록 화면에 출력되는게 많다. 
    두장을 그려서 합친다.
    계속 누적되는게 아니라 새로 그린다 ?
    두깨를 늘린다.
    이미지를 어떻게 합쳐 ?
    

